In [2]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install transformers[sentencepiece]
from google.colab import output
from huggingface_hub import notebook_login
from huggingface_hub import list_models, HfApi, ModelFilter
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

output.clear()

In [3]:
notebook_login()

In [4]:
list_models()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:229: FutureWarning: 'list_models' currently returns a list of objects but is planned to be a generator starting from version 0.14 in order to implement pagination. Please avoid to use `list_models(...).__len__` or explicitly convert the output to a list first with `list(iter(list_models)(...))`.
  warnings.warn(self._deprecation_msg.format(attr_name=attr_name), FutureWarning)


[ModelInfo: { 
   {'_id': '621ffdc036468d709f174328',
    'author': None,
    'config': None,
    'downloads': 54072,
    'id': 'albert-base-v1',
    'lastModified': None,
    'likes': 1,
    'modelId': 'albert-base-v1',
    'pipeline_tag': 'fill-mask',
    'private': False,
    'securityStatus': None,
    'sha': None,
    'siblings': [],
    'tags': ['pytorch', 'tf', 'safetensors', 'albert', 'fill-mask', 'en', 'dataset:bookcorpus', 'dataset:wikipedia',
             'arxiv:1909.11942', 'transformers', 'exbert', 'license:apache-2.0', 'autotrain_compatible', 'has_space']}
 },
 ModelInfo: { 
   {'_id': '621ffdc036468d709f174329',
    'author': None,
    'config': None,
    'downloads': 4499870,
    'id': 'albert-base-v2',
    'lastModified': None,
    'likes': 49,
    'modelId': 'albert-base-v2',
    'pipeline_tag': 'fill-mask',
    'private': False,
    'securityStatus': None,
    'sha': None,
    'siblings': [],
    'tags': ['pytorch', 'tf', 'jax', 'rust', 'safetensors', 'albert', 'fill

In [5]:
hf_api = HfApi()
models = hf_api.list_models(
    filter=ModelFilter(
        task=["text-generation", "conversational"]
    )
)

model_Ids = [m.modelId for m in models]

In [6]:
import ipywidgets as widgets

model_picker = widgets.Dropdown(options=model_Ids, value='microsoft/DialoGPT-medium')

In [7]:
model_name = model_picker.value
print(f"Selected model is: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name)

Selected model is: microsoft/DialoGPT-medium


In [8]:
step = 0
user_input = input(">> User:")
while user_input.lower()!='exit':
    
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step = step + 1
    user_input = input(">> User:")

>> User:exit
